# League of Legends Data Analysis
### My self-taught python journey

I've always been curious to drill deeper into the details of League of Legends performance than available through the standard platforms. As I teach myself Python, I want to dive deeper into what makes a Solo-queue winner: how do top performers compare the the rest of us in terms of damage share, consistency, and macro play?

## Step 0: Import Necessary Modules

In order to use any of this code, you'll need to get a key from https://developer.riotgames.com/. 

In [1]:
# @hidden_cell
key = 'removed per Riot guidelines'

In [2]:
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import requests
import json
import time
from datetime import datetime

In [3]:
url_Core = 'https://na1.api.riotgames.com'
url_Summ_By_Name = '/lol/summoner/v4/summoners/by-name/'
url_Match_History = '/lol/match/v4/matchlists/by-account/'
url_Match_Stats = '/lol/match/v4/matches/'
url_Match_Timeline = '/lol/match/v4/timelines/by-match/'

In [4]:
lookup_name = 'jigglemyjag'

In [5]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Step 1: Import Necessary Data

In [6]:
def summRequest(name):
    """Pulls identifying summoner information to be used in later methods.
    In: name
    Out: JSON dictionary"""
    url = f'{url_Core}{url_Summ_By_Name}{name}?api_key={key}'
    response = requests.get(url).json()
    #dataframe = json_normalize(response)
    return response

#### Explore the Match History Options 

In [7]:
json_normalize(summRequest(lookup_name))

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel
0,SBBIrs6Aue0Nxu-uCkGafeHvzh_Pf25okIMwQAro-ddkB4w,D6vSjql1vG4RsEXLbzFueyKjizX8tMm6HWfUFFpKand1kg,O5cnL8OBx44QpOC01b-5K36rxKW0jZkSJdK5GJBr6UBATv...,JiggleMyJag,4403,1578119924000,84


Note that we <b>can improve our summRequest function by addint startIndex and endIndex functionality

In [8]:
def getGameIdLst(name, champion=None, queue=None, lane=None):
    """Get a list of gameIds for the specified summoner, allowing for filters based on values for champion, queue, and lane.
    In: 
        champion: int
        queue: int
        lane: string
    Out:
        List of gameIds. Riot caps list at 100 max"""
    paramkey = {'champion': champion, 'queue': queue, 'api_key': key}
    acct_ID = summRequest(name)['accountId']
    url = f'{url_Core}{url_Match_History}{acct_ID}'
    response = requests.get(url, params=paramkey).json()
    lst = [match['gameId'] for match in response['matches'] if (lane == None or match['lane'].lower()==lane.lower())]
    if (len(lst)) == 0:
        print("No games meet these search results")
    return lst

In [9]:
def matchStats(matchID):
    """Accesses Riot's API for specific match stats, which requires a match Id
    In: gameId int
    Out: JSON dictionary"""
    url = f'{url_Core}{url_Match_Stats}{matchID}?api_key={key}'
    response = requests.get(url).json()
    return response

# Comparing List Comprehensions vs for loops
### When performing operations on large datasets, efficiency is key

In [10]:
def matchHStatsDataframe(name, champion=None, queue=None, lane=None):
    """returns a dictionary unpacking a game's match stats 
    In: list of gameId integers
    Out: dataframe match history with following headers"""
    lst = [matchStats(game) for game in getGameIdLst(name, champion, queue, lane)[:98]]
    df = pd.DataFrame.from_records(lst)
    return(df)

In [11]:
def matchHStatsDataframe_RateLimit(name, champion=None, queue=None, lane=None):
    """returns a dictionary unpacking a game's match stats, with options for pausing the code to accomodate rate limits.
    In: list of gameId integers
    Out: dataframe match history with following headers"""
    lst = []
    for i in getGameIdLst(name, champion, queue, lane)[:98]:
        #time.sleep(2) 
        #Pass the rate limiter requirement
        #Can make this step a list comprehension if this step becomes necessary
        lst.append(matchStats(i))
    df = pd.DataFrame.from_records(lst)
    return(df)

In [12]:
## we cannot use timeit because the loop creates exceptional overhead, so we will instead compare time 
time.sleep(125)       
start = datetime.now()
match_hist = matchHStatsDataframe(lookup_name)
print (f'List comprehension function takes {datetime.now()-start} seconds' )

time.sleep(125)       
start = datetime.now()
y = matchHStatsDataframe_RateLimit(name=lookup_name)
print (f'Rate Limit function takes {datetime.now()-start} seconds' )

List comprehension function takes 0:00:47.007812 seconds
Rate Limit function takes 0:00:46.574597 seconds


After running the above code multiple times, <strong>the difference in timing between the two functions is negligible</strong>.

We will use the list comprehension, as it is more pythonic.

# Unpacking Dictionaries within a Dataframe
### This was a very annoying process for a novice. The first listed solution I came up with on my own, the second was developed after checking multiple stackoverflow sollutions

#### Step 1: Data Exploration
Let's first explore the dataframe so you know what I'm talking about. While the first columns may appear standard, the final three contain nested lists of dictionaries - how annoying! Obviously we can make separate dataframes with these contents, but we would then lose our index variable (gameId)

In [13]:
match_hist.head(2)

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,teams,participants,participantIdentities
0,3259063988,NA1,1578631190237,1171,450,12,13,10.1.303.9385,ARAM,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
1,3258355800,NA1,1578545344179,1825,440,11,13,10.1.303.9385,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."


#### Step 2: Unpacking

##### 2.1 Attempt One

Here is my first attempt at tackling this problem. Coming from an SQL background, I resolved to simply make a bajillion tables and join them together on similar keys (presumably gameId and participantId). 

It's messy, but it (technically) gets the job done

In [14]:
def unpackNestedObjs(df, attribute):
    """Unpacks nested dictionaries and lists of lists within dataframes. This method gets replaced further on"""
    temp = df
    lst = []
    count = 0
    if attribute in ['participants', 'participantIdentities', 'teams']:
        for key, row in temp.iterrows():
            count += 1
            try:
                for value in row[attribute]:
                    value['gameId'] = row.gameId #key # gameId
                    lst.append(value)
            except:
                print(f'error at game number {count}')
    else:
        for key, row in temp.iterrows():
            if 'participantId' not in row[attribute].keys():
                row[attribute]['participantId'] = row.participantId
            row[attribute]['gameId'] = key
            lst.append(row[attribute])
    response = pd.DataFrame.from_records(lst, index='gameId')                   #indexing off of gameId messes up calculation. find out why.
    #response.columns = ['participantId' if x == 'summonerId' else x for x in response.columns]
    return response

In [15]:
unpackNestedObjs(match_hist , 'participants')[:20:6]

,participantId,teamId,championId,spell1Id,spell2Id,stats,timeline
gameId,,,,,,,
3259063988,1,100,45,4,13,"{'participantId': 1, 'win': False, 'item0': 24...","{'participantId': 1, 'creepsPerMinDeltas': {'0..."
3259063988,7,200,55,32,4,"{'participantId': 7, 'win': True, 'item0': 302...","{'participantId': 7, 'creepsPerMinDeltas': {'0..."
3258355800,3,100,63,4,14,"{'participantId': 3, 'win': False, 'item0': 31...","{'participantId': 3, 'creepsPerMinDeltas': {'1..."
3258355800,9,200,84,4,14,"{'participantId': 9, 'win': True, 'item0': 242...","{'participantId': 9, 'creepsPerMinDeltas': {'1..."


You'll note that that none the data from the original Match History Dateframe is captured in this new dataframe barring the gameId variable, and that was only possible through adding multiple lines of code to add it conditionally. There must be a better way...

###### Attempt 2

Adding a multiindex with gameId preserves the original positional index and gameId when performing more granular analysis. In the example below, gameId becomes the left index, while the original numeric index becomes the right. 

In [16]:
idx = match_hist.set_index('gameId').participants.apply(pd.Series).stack().index
idx[:5]

MultiIndex([(3259063988, 0),
            (3259063988, 1),
            (3259063988, 2),
            (3259063988, 3),
            (3259063988, 4)],
           names=['gameId', None])

##### 2.1 hard-coded variables
So long as Riot keeps their data in a stable format, this should work. It does look a little messy, but it works well

In [17]:
def unpackNestedObjs2(df):
    """Unpacks nested dictionaries within the participants, participantIdentities, and teams columns using a multiIndex w/GameId.
    This is accomplished by calling the column contents and converting the column values from a series to a dictionary.
    After the unpacking is done, deletes the original column """
    #unpack participants   
    idx = df.set_index('gameId')['participants'].apply(pd.Series).stack().index 
    participants = pd.DataFrame(df['participants'].apply(pd.Series).stack().values.tolist())
    frame = df.merge(participants, on=['gameId'], how='left')
    #unpack participantIdentities
    idx = df.set_index('gameId')['participantIdentities'].apply(pd.Series).stack().index
    ids = pd.DataFrame(df.participantIdentities.apply(pd.Series).stack().values.tolist(), index=idx).reset_index().drop('level_1', 1)
    frame = frame.merge(ids, on = ['gameId', 'participantId'], how ='left')
    #unpack teams
    idx = df.set_index('gameId')['teams'].apply(pd.Series).stack().index
    teams = pd.DataFrame(df.teams.apply(pd.Series).stack().values.tolist(), index=idx).reset_index().drop('level_1', 1)
    frame = frame.merge(teams, on = ['gameId', 'teamId'], how ='left')

    frame.drop(columns=['participants', 'participantIdentities', 'teams'], inplace=True)

    for i in ['stats', 'timeline', 'player']:
        frame = frame.drop(i, 1).assign(**pd.DataFrame.from_records(frame[i].dropna().tolist()))
        
    frame.drop(columns = ['combatPlayerScore', 'objectivePlayerScore','totalPlayerScore', 'totalScoreRank', 
    'playerScore0', 'playerScore1', 'playerScore2', 'playerScore3','playerScore4', 
    'playerScore5', 'playerScore6', 'playerScore7', 'playerScore8', 'playerScore9'], inplace=True)
    
    return frame

Let's make sure the output looks right: Yup, it does. <p> 
    
After scanning the dataframe, there are some columns that look like they're not used: <em> combatPlayerScore	objectivePlayerScore	totalPlayerScore	totalScoreRank	playerScore0	playerScore1	playerScore2	playerScore3	playerScore4	playerScore5	playerScore6	playerScore7	playerScore8	playerScore9. </em> <p>
    
I've gone ahead and added a line to remove the in the code below.

In [18]:
unpackNestedObjs2(match_hist)[:15:3]

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,participantId,teamId,championId,spell1Id,spell2Id,win,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,towerKills,inhibitorKills,baronKills,dragonKills,vilemawKills,riftHeraldKills,dominionVictoryScore,bans,item0,item1,item2,item3,item4,item5,item6,kills,deaths,assists,largestKillingSpree,largestMultiKill,killingSprees,longestTimeSpentLiving,doubleKills,tripleKills,quadraKills,pentaKills,unrealKills,totalDamageDealt,magicDamageDealt,physicalDamageDealt,trueDamageDealt,largestCriticalStrike,totalDamageDealtToChampions,magicDamageDealtToChampions,physicalDamageDealtToChampions,trueDamageDealtToChampions,totalHeal,totalUnitsHealed,damageSelfMitigated,damageDealtToObjectives,damageDealtToTurrets,visionScore,timeCCingOthers,totalDamageTaken,magicalDamageTaken,physicalDamageTaken,trueDamageTaken,goldEarned,goldSpent,turretKills,totalMinionsKilled,neutralMinionsKilled,totalTimeCrowdControlDealt,champLevel,visionWardsBoughtInGame,sightWardsBoughtInGame,firstBloodKill,firstBloodAssist,firstTowerKill,firstTowerAssist,firstInhibitorKill,firstInhibitorAssist,perk0,perk0Var1,perk0Var2,perk0Var3,perk1,perk1Var1,perk1Var2,perk1Var3,perk2,perk2Var1,perk2Var2,perk2Var3,perk3,perk3Var1,perk3Var2,perk3Var3,perk4,perk4Var1,perk4Var2,perk4Var3,perk5,perk5Var1,perk5Var2,perk5Var3,perkPrimaryStyle,perkSubStyle,statPerk0,statPerk1,statPerk2,neutralMinionsKilledTeamJungle,neutralMinionsKilledEnemyJungle,wardsPlaced,wardsKilled,creepsPerMinDeltas,xpPerMinDeltas,goldPerMinDeltas,csDiffPerMinDeltas,xpDiffPerMinDeltas,damageTakenPerMinDeltas,damageTakenDiffPerMinDeltas,role,lane,accountId,summonerName,summonerId,currentPlatformId,currentAccountId,matchHistoryUri,profileIcon
0,3259063988,NA1,1578631190237,1171,450,12,13,10.1.303.9385,ARAM,MATCHED_GAME,1,100,45,4,13,False,False,False,False,False,False,False,1,0,0,0,0,0,0,[],2424,3285,3020,3089,3165,0,2052,8,14,13,2,2,2,109,1,0,0,0,0,69387,67966,1421,0,0,19884,19488,395,0,543,1,5772,77,77,0,33,21323,16363,4187,772,11523,11000,0,48,0,92,17,0,0,False,False,False,False,False,False,8229,1427,0,0,8226,250,681,0,8210,0,0,0,8236,48,0,0,8313,0,0,0,8347,0,0,0,8200,8300,5007,5008,5001,NaN,NaN,NaN,NaN,{'0-10': 1.9000000000000001},{'0-10': 647.2},{'0-10': 437},{'0-10': -0.2799999999999998},{'0-10': -28.19999999999999},{'0-10': 931},{'0-10': 108.15999999999997},DUO_SUPPORT,NONE,1PFEUGMi1YdwYp19b57GDEQ5G8L294jRgzXjWv8hqBjx7r...,ColtonFox 2,FZ4yEL0dtelv8lejemVM6eIfm-K2YfUD7lhc5LU9x8WX2rPy,NA1,1PFEUGMi1YdwYp19b57GDEQ5G8L294jRgzXjWv8hqBjx7r...,/v1/stats/player_history/NA1/2352143649933984,4449
3,3259063988,NA1,1578631190237,1171,450,12,13,10.1.303.9385,ARAM,MATCHED_GAME,4,100,103,7,4,False,False,False,False,False,False,False,1,0,0,0,0,0,0,[],3048,3116,3151,3020,0,0,2052,5,7,24,2,2,1,329,1,0,0,0,0,23531,14542,4612,4376,0,10939,8472,1208,1258,3431,5,8235,0,0,0,19,17879,9602,7362,914,10681,10350,0,18,0,91,17,0,0,False,False,False,False,False,False,8128,1231,20,693,8143,93,0,0,8138,30,0,0,8106,5,0,0,8275,7,0,0,8237,356,0,0,8100,8200,5008,5003,5003,NaN,NaN,NaN,NaN,{'0-10': 0.8999999999999999},{'0-10': 750.3},{'0-10': 447.4},{'0-10': -0.2799999999999998},{'0-10': -28.19999999999999},{'0-10': 672.1},{'0-10': 108.15999999999997},DUO_SUPPORT,NONE,yAkZC_49mwkR6F9xyRivtj6ZEJkMnhPV4GtgXQGv68AkER...,irisesta,eOs3cuZTcCGbjBXj6fwIgRogCRQsLR3XtpPggT4accQCdQua,NA1,yAkZC_49mwkR6F9xyRivtj6ZEJkMnhPV4GtgXQGv68AkER...,/v1/stats/player_history/NA1/2301122858723648,4367
6,3259063988,NA1,1578631190237,1171,450,12,13,10.1.303.9385,ARAM,MATCHED_GAME,7,200,55,32,4,True,True,True,True,False,False,False,4,0,0,0,0,0,0,[],3020,3146,3116,3089,1026,1052,2052,17,12,17,4,4,5,142,4,2,1,0,0,58874,54231,3327,1314,0,29501,26742,1444,1314,4392,1,15557,589,589,0,5,26247,5815,18534,1897,12942,11985,0,34,0,71,18,0,0,True,False,False,False,False,False,8112,1586,0,0,8143,873,0,0,8138,30,0,0,8135,1835,5,0,9111,613,340,0,8014,838,0,0,8100,8000,5008,

##### 2.2 Loops! 
Reformatting this code into a loopable format will be helpful if I ever need to use it again for another dataframe. Let's give it a shot:

In [19]:
def unpackNestedObjs_Loop(df):
    """Unpacks nested dictionaries within the participants, participantIdentities, and teams columns using a multiIndex w/GameId.
    This is accomplished by calling the column contents and converting the column values from a series to a dictionary.
    After the unpacking is done, deletes the original column """
    frame = df
    cols = ['participants', 'participantIdentities', 'teams']
    indexes = [['gameId'], ['gameId', 'participantId'], ['gameId', 'teamId']]
    for col, index in zip(cols, indexes):
        idx = df.set_index('gameId')[col].apply(pd.Series).stack().index
        if (len(index) == 1):
            temp = pd.DataFrame(df[col].apply(pd.Series).stack().values.tolist())
        else:
            temp = pd.DataFrame(df[col].apply(pd.Series).stack().values.tolist(), index = idx).reset_index().drop('level_1', 1)
        frame = frame.merge(temp, on = index, how = 'left')
        
    frame.drop(columns=cols, inplace=True)    
    for i in ['stats', 'timeline', 'player']:
        frame = frame.drop(i, 1).assign(**pd.DataFrame.from_records(frame[i].dropna().tolist()))
    
    frame.drop(columns = ['combatPlayerScore', 'objectivePlayerScore','totalPlayerScore', 'totalScoreRank', 
    'playerScore0', 'playerScore1', 'playerScore2', 'playerScore3','playerScore4', 
    'playerScore5', 'playerScore6', 'playerScore7', 'playerScore8', 'playerScore9'], inplace=True)
               
    return frame

##### A question for those more familiar with Python: why do I need to convert values.tolist() to make the above function work? It seems to construct single dataframes just fine in isolation, but once I put it the combined function it gives me a key error...

I actually do not know 

Let's quickly confirm that both methods return the same output:

In [20]:
unpackNestedObjs_Loop(match_hist).equals(unpackNestedObjs2(match_hist))

True

Now that we have two working methods, let's make sure there aren't any performance issues:

In [21]:
%timeit unpackNestedObjs2(match_hist)
%timeit unpackNestedObjs_Loop(match_hist)

419 ms ± 27.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
425 ms ± 25.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<b> Does the loop work? </b> Yes. <p>
<b> Is it faster? </b> A bit, yes. <p>
I'll be moving forward with the loop method as it's easier to follow. 

In [22]:
master = unpackNestedObjs_Loop(match_hist)
master.head(2)

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,participantId,teamId,championId,spell1Id,spell2Id,win,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,towerKills,inhibitorKills,baronKills,dragonKills,vilemawKills,riftHeraldKills,dominionVictoryScore,bans,item0,item1,item2,item3,item4,item5,item6,kills,deaths,assists,largestKillingSpree,largestMultiKill,killingSprees,longestTimeSpentLiving,doubleKills,tripleKills,quadraKills,pentaKills,unrealKills,totalDamageDealt,magicDamageDealt,physicalDamageDealt,trueDamageDealt,largestCriticalStrike,totalDamageDealtToChampions,magicDamageDealtToChampions,physicalDamageDealtToChampions,trueDamageDealtToChampions,totalHeal,totalUnitsHealed,damageSelfMitigated,damageDealtToObjectives,damageDealtToTurrets,visionScore,timeCCingOthers,totalDamageTaken,magicalDamageTaken,physicalDamageTaken,trueDamageTaken,goldEarned,goldSpent,turretKills,totalMinionsKilled,neutralMinionsKilled,totalTimeCrowdControlDealt,champLevel,visionWardsBoughtInGame,sightWardsBoughtInGame,firstBloodKill,firstBloodAssist,firstTowerKill,firstTowerAssist,firstInhibitorKill,firstInhibitorAssist,perk0,perk0Var1,perk0Var2,perk0Var3,perk1,perk1Var1,perk1Var2,perk1Var3,perk2,perk2Var1,perk2Var2,perk2Var3,perk3,perk3Var1,perk3Var2,perk3Var3,perk4,perk4Var1,perk4Var2,perk4Var3,perk5,perk5Var1,perk5Var2,perk5Var3,perkPrimaryStyle,perkSubStyle,statPerk0,statPerk1,statPerk2,neutralMinionsKilledTeamJungle,neutralMinionsKilledEnemyJungle,wardsPlaced,wardsKilled,creepsPerMinDeltas,xpPerMinDeltas,goldPerMinDeltas,csDiffPerMinDeltas,xpDiffPerMinDeltas,damageTakenPerMinDeltas,damageTakenDiffPerMinDeltas,role,lane,accountId,summonerName,summonerId,currentPlatformId,currentAccountId,matchHistoryUri,profileIcon
0,3259063988,NA1,1578631190237,1171,450,12,13,10.1.303.9385,ARAM,MATCHED_GAME,1,100,45,4,13,False,False,False,False,False,False,False,1,0,0,0,0,0,0,[],2424,3285,3020,3089,3165,0,2052,8,14,13,2,2,2,109,1,0,0,0,0,69387,67966,1421,0,0,19884,19488,395,0,543,1,5772,77,77,0,33,21323,16363,4187,772,11523,11000,0,48,0,92,17,0,0,False,False,False,False,False,False,8229,1427,0,0,8226,250,681,0,8210,0,0,0,8236,48,0,0,8313,0,0,0,8347,0,0,0,8200,8300,5007,5008,5001,NaN,NaN,NaN,NaN,{'0-10': 1.9000000000000001},{'0-10': 647.2},{'0-10': 437},{'0-10': -0.2799999999999998},{'0-10': -28.19999999999999},{'0-10': 931},{'0-10': 108.15999999999997},DUO_SUPPORT,NONE,1PFEUGMi1YdwYp19b57GDEQ5G8L294jRgzXjWv8hqBjx7r...,ColtonFox 2,FZ4yEL0dtelv8lejemVM6eIfm-K2YfUD7lhc5LU9x8WX2rPy,NA1,1PFEUGMi1YdwYp19b57GDEQ5G8L294jRgzXjWv8hqBjx7r...,/v1/stats/player_history/NA1/2352143649933984,4449
1,3259063988,NA,1578631190237,1171,450,12,13,10.1.303.9385,ARAM,MATCHED_GAME,2,100,11,4,32,False,False,False,False,False,False,False,1,0,0,0,0,0,0,[],3087,3031,3006,3046,3052,1028,2052,13,10,20,4,3,2,177,1,1,0,0,0,72384,8688,62252,1443,517,22384,1790,19360,1233,6526,1,18247,638,638,0,0,24254,11657,11967,628,13000,11400,0,55,0,57,18,0,0,False,False,False,False,False,False,8010,705,0,0,9111,812,330,0,9104,5,20,0,8014,630,0,0,8139,907,0,0,8135,1736,5,0,8000,8100,5005,5008,5003,NaN,NaN,NaN,NaN,{'0-10': 2.4},{'0-10': 746.8},{'0-10': 591.4000000000001},{'0-10': -0.2799999999999998},{'0-10': -28.19999999999999},{'0-10': 1015.3},{'0-10': 108.15999999999997},DUO_SUPPORT,NONE,y4Lmk-e-sdXosrHXXChDAhRl2DJOKe2V5gnuRD8mjYLkug,DingusFox,flI8sB1rHz9ziOj92nHxyiRWc5QE_esBVKCZMGRUVoOhk4k,NA1,y4Lmk-e-sdXosrHXXChDAhRl2DJOKe2V5gnuRD8mjYLkug,/v1/stats/player_history/NA/37135223,4404


It would certainly look nicer if <strong> both </strong> of our index columns werein the front, wouldn't it? </p>

I'm sure there's way to to this that doesn't involve temporary reindexing or making a reordered column list with every individual column, but considering this master data frame has so many columns, this is the fastest way I could think of: 

In [23]:
df = master
df.set_index(['gameId', 'summonerName', 'participantId'], inplace=True)
df.reset_index(inplace=True)
df.head(2)

,gameId,summonerName,participantId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,teamId,championId,spell1Id,spell2Id,win,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,towerKills,inhibitorKills,baronKills,dragonKills,vilemawKills,riftHeraldKills,dominionVictoryScore,bans,item0,item1,item2,item3,item4,item5,item6,kills,deaths,assists,largestKillingSpree,largestMultiKill,killingSprees,longestTimeSpentLiving,doubleKills,tripleKills,quadraKills,pentaKills,unrealKills,totalDamageDealt,magicDamageDealt,physicalDamageDealt,trueDamageDealt,largestCriticalStrike,totalDamageDealtToChampions,magicDamageDealtToChampions,physicalDamageDealtToChampions,trueDamageDealtToChampions,totalHeal,totalUnitsHealed,damageSelfMitigated,damageDealtToObjectives,damageDealtToTurrets,visionScore,timeCCingOthers,totalDamageTaken,magicalDamageTaken,physicalDamageTaken,trueDamageTaken,goldEarned,goldSpent,turretKills,totalMinionsKilled,neutralMinionsKilled,totalTimeCrowdControlDealt,champLevel,visionWardsBoughtInGame,sightWardsBoughtInGame,firstBloodKill,firstBloodAssist,firstTowerKill,firstTowerAssist,firstInhibitorKill,firstInhibitorAssist,perk0,perk0Var1,perk0Var2,perk0Var3,perk1,perk1Var1,perk1Var2,perk1Var3,perk2,perk2Var1,perk2Var2,perk2Var3,perk3,perk3Var1,perk3Var2,perk3Var3,perk4,perk4Var1,perk4Var2,perk4Var3,perk5,perk5Var1,perk5Var2,perk5Var3,perkPrimaryStyle,perkSubStyle,statPerk0,statPerk1,statPerk2,neutralMinionsKilledTeamJungle,neutralMinionsKilledEnemyJungle,wardsPlaced,wardsKilled,creepsPerMinDeltas,xpPerMinDeltas,goldPerMinDeltas,csDiffPerMinDeltas,xpDiffPerMinDeltas,damageTakenPerMinDeltas,damageTakenDiffPerMinDeltas,role,lane,accountId,summonerId,currentPlatformId,currentAccountId,matchHistoryUri,profileIcon
0,3259063988,ColtonFox 2,1,NA1,1578631190237,1171,450,12,13,10.1.303.9385,ARAM,MATCHED_GAME,100,45,4,13,False,False,False,False,False,False,False,1,0,0,0,0,0,0,[],2424,3285,3020,3089,3165,0,2052,8,14,13,2,2,2,109,1,0,0,0,0,69387,67966,1421,0,0,19884,19488,395,0,543,1,5772,77,77,0,33,21323,16363,4187,772,11523,11000,0,48,0,92,17,0,0,False,False,False,False,False,False,8229,1427,0,0,8226,250,681,0,8210,0,0,0,8236,48,0,0,8313,0,0,0,8347,0,0,0,8200,8300,5007,5008,5001,NaN,NaN,NaN,NaN,{'0-10': 1.9000000000000001},{'0-10': 647.2},{'0-10': 437},{'0-10': -0.2799999999999998},{'0-10': -28.19999999999999},{'0-10': 931},{'0-10': 108.15999999999997},DUO_SUPPORT,NONE,1PFEUGMi1YdwYp19b57GDEQ5G8L294jRgzXjWv8hqBjx7r...,FZ4yEL0dtelv8lejemVM6eIfm-K2YfUD7lhc5LU9x8WX2rPy,NA1,1PFEUGMi1YdwYp19b57GDEQ5G8L294jRgzXjWv8hqBjx7r...,/v1/stats/player_history/NA1/2352143649933984,4449
1,3259063988,DingusFox,2,NA,1578631190237,1171,450,12,13,10.1.303.9385,ARAM,MATCHED_GAME,100,11,4,32,False,False,False,False,False,False,False,1,0,0,0,0,0,0,[],3087,3031,3006,3046,3052,1028,2052,13,10,20,4,3,2,177,1,1,0,0,0,72384,8688,62252,1443,517,22384,1790,19360,1233,6526,1,18247,638,638,0,0,24254,11657,11967,628,13000,11400,0,55,0,57,18,0,0,False,False,False,False,False,False,8010,705,0,0,9111,812,330,0,9104,5,20,0,8014,630,0,0,8139,907,0,0,8135,1736,5,0,8000,8100,5005,5008,5003,NaN,NaN,NaN,NaN,{'0-10': 2.4},{'0-10': 746.8},{'0-10': 591.4000000000001},{'0-10': -0.2799999999999998},{'0-10': -28.19999999999999},{'0-10': 1015.3},{'0-10': 108.15999999999997},DUO_SUPPORT,NONE,y4Lmk-e-sdXosrHXXChDAhRl2DJOKe2V5gnuRD8mjYLkug,flI8sB1rHz9ziOj92nHxyiRWc5QE_esBVKCZMGRUVoOhk4k,NA1,y4Lmk-e-sdXosrHXXChDAhRl2DJOKe2V5gnuRD8mjYLkug,/v1/stats/player_history/NA/37135223,4404


# Section 3: Analysis
Now that we've got our data wrangled into a master frame, we can get to our analysis. </p>

Let's identifiy some of our goals:
- How much damage does contribute summoner contribute in victories vs defeats?
- Kill and Death locations in wins vs losses

#### Analysis 3.1 Damage Share

In [24]:
def WinLossDamageShare(name, aggregated_df, lane=None):
    """Displays the average total damage done by the summoner, grouped by wins and losses.
        Win: False indicates stats in a loss.
        Win: True indicates stats in a win"""
    summ_filter = master['summonerName'].str.lower() == lookup_name.lower()             #str.lower() for dataframes
    lst = [(row.gameId, row.teamId) for key, row in master[summ_filter].iterrows()]
    df_lst = [row for key, row in master.iterrows() if ((row.gameId, row.teamId) in lst)]
    dfallies = pd.DataFrame(df_lst)

    #print messages
    print(f"{name}'s Damage Analsysis: Raw Numbers")
    print(aggregated_df[summ_filter].groupby('win').agg({'totalDamageDealtToChampions': [np.mean, np.size]}))
    print('\n\n %Team Damage Share in Wins and Losses')
    print( aggregated_df[summ_filter].groupby('win').sum().totalDamageDealtToChampions /
    dfallies.groupby('win').sum().totalDamageDealtToChampions)

    return

The below cell displays the average total damage done by the summoner (using the specified queue and champion filters from earlier) by wins and losses. 

<p> <b> Win: False</b> indicates stats in a loss. </p>
<p> <b> Win: True</b> indicates stats in a win </p>

In [25]:
WinLossDamageShare('JiggleMyJag', aggregated_df=master)

JiggleMyJag's Damage Analsysis: Raw Numbers
      totalDamageDealtToChampions     
                             mean size
win                                   
False                22321.434783   46
True                 23015.461538   52


 %Team Damage Share in Wins and Losses
win
False    0.222281
True     0.230344
Name: totalDamageDealtToChampions, dtype: float64


# That's it for now - check in later for more analysis and data vizualization :) 